# Script for unziping compressed sub files

In [8]:
import os, sys, re
import zipfile as zf
import shutil

In [7]:
def unzip_rename(f_path=os.getcwd()):
    """Move subs zip files to correct folders, 
    unpack & rename to video filename."""
    
    # if such a file/folder exists
    if os.path.exists(f_path) == True:
        print('Path: ' + f_path + ' exists.')
    else:
        print('There is no such directory: ' + path)
        return 0
    
    # regex patterns
    start_p = re.compile(r'^19\d{2}--|^20\d{2}--')
    year_p = re.compile(r"19\d{2}|20\d{2}")
    name_p = re.compile(r"^(?=.*\d{4})(?!.*\.zip)[A-Za-z\s\.'-]+[^\(\.\s\-]\b", re.I)
    sep_p = re.compile(r"\s|\.|\_|\-")
    
    # rename movie folders if not already
    for file in os.listdir(f_path): 
        
        # skip file if it is already renamed
        if name_p.match(file) is not None:
            
            # extracts the year & name from the file
            year = re.search(year_p, file)
            name_raw = re.match(name_p, file)
            name = re.sub(sep_p, ' ', name_raw[0].lower())
            
            if os.path.exists(f"{f_path}/{file}") == True:
                os.rename(f"{f_path}/{file}", f"{f_path}/{year[0]}--{name}")
        
    """"""
    # regex patterns
    mfile_p = re.compile(r"^(?=.*\.mp4$|.*\.avi$|.*\.mkv$).*")
    mfile_ext = re.compile(r"\.(mp4|.*avi|.*mkv)$")
    subs_folder_p = re.compile(r"^Subs$", re.I)
    eng_p = re.compile(r"(?i)eng|EN")
    sub_p = re.compile(r"\.srt$")
    zip_p = re.compile(r"^.*\.zip")
    zip_name_p = re.compile(r"^(?=.*\.zip)[A-Za-z\s\.'-]+[^\(\.\s\-]\b")
    
    # rename movie & sub files in folders to same name
    for file in os.listdir(f_path):
        if start_p.match(file) is not None:
            for dir_file in os.listdir(f"{f_path}/{file}"):
                
                if mfile_p.match(dir_file) is not None:
                    ext = mfile_ext.search(dir_file)
                    os.rename(f"{f_path}/{file}/{dir_file}", 
                              f"{f_path}/{file}/{file}{ext}")
                    
                if sub_p.search(dir_file) is not None:
                    if f"{f_path}/{file}/{file}.en.srt"  
                       f"{f_path}/{file}/{file}.srt"
                    
                    os.rename(f"{f_path}/{file}/{dir_file}",
                              f"{f_path}/{file}/{file}.en.srt")
                    shutil.copy(f"{f_path}/{file}/{file}.en.srt",
                                f"{f_path}/{file}/{file}.srt")
                    
                if subs_folder_p.match(dir_file) is not None:
                    for sub in os.listdir(f"{f_path}/{file}/{dir_file}"):
                        if eng_p.search(subs) is not None:
                            os.rename(f"{f_path}/{file}/{dir_file}/{sub}",
                                      f"{f_path}/{file}/{file}.en.srt")
                            shutil.copy(f"{f_path}/{file}/{file}.en.srt",
                                f"{f_path}/{file}/{file}.srt")

#     # move zip file to corresponding folder
#     for file in os.listdir(f_path):
        
#         if zip_p.match(file):
            
#             print(file)
#             # extract movie name
#             year = re.search(year_p, file)
#             name_raw = re.match(zip_name_p, file)
#             name = re.sub(sep_p, ' ', name_raw[0].lower())
            
#             movie_dir = f"{year[0]}--{name}"
            
#             # cut zip file to movie folder if not already
#             if not os.path.exists(f"{f_path}/{movie_dir}/{file}"):
#                 os.rename(f"{f_path}/{file}", f"{f_path}/{movie_dir}/{file}")
                
#                 # unzip file
#                 with zf.ZipFile(f"{f_path}/{movie_dir}/{file}", 'r') as zip_ref:
#                     zip_ref.extractall(f"{f_path}/{movie_dir}")
                
#             else:
#                 os.remove(f"{f_path}/{file}")

unzip_rename('/home/tonu/Downloads')

Path: /home/tonu/Downloads exists.
Everything.Is.Illuminated.2005.720P.Webrip.X264.[Moviesfd].mkv
The Big Lebowski (1998) (1080p BluRay x265 HDR afm72).mkv
Contact.1997.1080p.BluRay.x264.YIFY.mp4
[ www.UsaBit.com ] - The Boat That Rocked 2009 720p BRRip x264-PLAYNOW.mp4
Harry_Potter_and_the_Half_Blood_Prince_2009.mkv
Nights.Of.Cabiria.1957.1080p.BluRay.x264.AAC-[YTS.MX].mp4
The.Polar.Express.2004.1080p.BluRay.x265-RARBG.mp4
The Impossible (2012) 720p_BRrip_scOrp_sujaidr.mkv
The.Wind.Will.Carry.Us.1999.1080p.BluRay.x264-[YTS.LT].mp4
Long.Day's.Journey.Into.Night.2018.1080p.BluRay.x264.AAC5.1-[YTS.MX].mp4
Red River (1948) Criterion (1080p BluRay x265 10bit Tigole).mkv
Breathe.2014.1080p.BluRay.x264.AAC5.1-[YTS.MX].mp4
The.Grey.2011.1080p.BluRay.x264.x264.YIFY.mp4
Kon-Tiki.2012.1080p.BluRay.x264-[YTS.LT].mp4
Ikiru.1952.1080p.BluRay.x264-[YTS.AM].mp4


In [6]:
pat = re.compile(r'^19\d{2}--|^20\d{2}--')
for f in os.listdir('/home/tonu/Downloads'):
    for fi in os.listdir(f'/home/tonu/Downloads/{f}'):
        print(fi)
    break

Everything.Is.Illuminated.2005.720P.Webrip.X264.[Moviesfd].mkv


In [11]:
"Ikiru.1952.1080p.BluRay.x264-[YTS.AM].srt" in os.listdir('/home/tonu/Downloads/1952--ikiru/')

True